# Download data from ERDDAP and plot with Plotly

In [1]:
import pandas as pd
import numpy as np
import os

# from _plotly_future_ import v4_subplots
import plotly
# import chart_studio.plotly as py
import plotly.graph_objs as go
from plotly import tools
import datetime

from erddapy import ERDDAP

### Step 1: access data on ERDDAP
- from this URL: http://erddap.sccoos.org/erddap/tabledap/pH-AHL.csv
- put into DataFrame

In [16]:
e = ERDDAP(
  server="http://erddap.sccoos.org/erddap",
  protocol='tabledap',
)
e.response = 'csv'
e.dataset_id = 'autoss'

e.constraints = {
    'time>=': '2017-09-01T00:00:00Z',
    'time<=': '2019-08-31T00:00:00Z',
#     'latitude>=': 38.0,
#     'latitude<=': 41.0,
#     'longitude>=': -72.0,
#     'longitude<=': -69.0,
}
e.variables = [
    'station',
    'depth',
    'pressure',
#     'latitude',
#     'longitude',
    'salinity',
    'temperature',
    'time',
]

df_all_sccoos = e.to_pandas()

# Trim a bit for plotting speed
# df_ahl_seaphox = df_ahl_seaphox.iloc[::2, :]
df_all_sccoos.head()

,station,depth (m),pressure,salinity,temperature (celsius),time (UTC)
0,scripps_pier,5.0,3.990,33.3556,16.8593,2017-09-01T00:02:54Z
1,scripps_pier,5.0,4.087,33.3170,16.5745,2017-09-01T00:06:05Z
2,scripps_pier,5.0,4.094,33.3055,17.0382,2017-09-01T00:10:54Z
3,scripps_pier,5.0,4.051,33.3286,16.5041,2017-09-01T00:14:05Z
4,scripps_pier,5.0,4.025,33.3237,16.4571,2017-09-01T00:18:54Z


### Find out what stations are included; filter for Scripps Pier

In [17]:
print("All stations in original DF: ", df_all_sccoos.loc[:, 'station'].unique())
df_scrippspier = df_all_sccoos[df_all_sccoos.loc[:, 'station']=='scripps_pier']
print("Station(s) in filtered DF:", df_scrippspier.loc[:, 'station'].unique())
print("Columns of filtered DF:", df_scrippspier.columns)

All stations in original DF:  ['scripps_pier' 'stearns_wharf' 'newport_pier' '005_newport_pier'
 'santa_monica_pier']
Station(s) in filtered DF: ['scripps_pier']
Columns of filtered DF: Index(['station', 'depth (m)', 'pressure', 'salinity', 'temperature (celsius)',
       'time (UTC)'],
      dtype='object')


### Make Plotly subplots; save HTML output

In [21]:
# Create traces
trace1 = go.Scatter(
    x = df_scrippspier['time (UTC)'], 
    y = df_scrippspier['temperature (celsius)'],
    mode = 'lines',
    name = 'Temperature'
)

trace2 = go.Scatter(
    x = df_scrippspier['time (UTC)'], 
    y = df_scrippspier['salinity'],
    mode = 'lines',
    name = 'Salinity'
)


# data = [trace1, trace2]

fig = plotly.subplots.make_subplots(rows = 2, cols = 1, shared_xaxes=True)

fig.append_trace(trace1, 1, 1)
fig.append_trace(trace2, 2, 1)

# Update yaxis properties
fig.update_yaxes(title_text="Temp (ºC)", row=1, col=1, range=[10, 27])
fig.update_yaxes(title_text="Sal (PSU)", row=2, col=1, range=[32.5, 34.5])

# fig.update_xaxes(title_text="Time (UTC)", row=2, col=1, range=[datetime.datetime(2019, 8, 22),
#                                                        max(df_ahl_seaphox['time (UTC)'])])

# Plot and embed here
# py.iplot(fig, filename='ahl_subplots.html')
plotly.offline.plot(fig, filename='../scrippspier_subplots.html')

'../scrippspier_subplots.html'

In [19]:
df_scrippspier.to_csv('Temp-Salt_ERDDAP_Scripps-Pier.csv')